In [0]:
df = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/FileStore/sample_data/dataset-1.csv")
df.show()


+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|     null|           8|        6|      0|               2|        2|            14|
|     null|           9|        8|      3|               0|        0|            14|
|     null|          10|        9|      2|               0|        1|            14|
|     null|          11|       11|      1|               4|        4|            11|
|     null|          12|       12|      0|               2|        2|            11|
|     null|          13|        9|      1|               0|        0|             9|
|     null|          14|       12|      1|               0|        0|             9|
|     null|          15|       11|      2|               1|      

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a SparkSession
spark = SparkSession.builder.appName("UberDataAnalysis").getOrCreate()

# Load the dataset into a DataFrame
df = spark.read.csv("dbfs:/FileStore/sample_data/dataset-1.csv", header=True, inferSchema=True) 

In [0]:
from pyspark.sql.functions import col

# Clean spaces in all columns
uber = uber.select([col(c).alias(c.strip()) for c in uber.columns])


In [0]:
uber.select("Eyeballs", "Date").show()


+--------+---------+
|Eyeballs|     Date|
+--------+---------+
|       5|10-Sep-12|
|       6|     null|
|       8|     null|
|       9|     null|
|      11|     null|
|      12|     null|
|       9|     null|
|      12|     null|
|      11|     null|
|      11|     null|
|      12|     null|
|      11|     null|
|      13|     null|
|      11|     null|
|      11|     null|
|      16|     null|
|      21|     null|
|       9|11-Sep-12|
|       3|     null|
|       1|     null|
+--------+---------+
only showing top 20 rows



In [0]:
df_filtered = df.filter(col("Date") != "null")
df_filtered.show()



+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|11-Sep-12|           0|        9|      3|               1|        1|             3|
|12-Sep-12|           0|        9|      2|               0|        1|             2|
|13-Sep-12|           0|       11|     11|               0|        2|             0|
|14-Sep-12|           0|       10|      1|               3|        4|             3|
|15-Sep-12|           0|       45|      2|              23|       24|            19|
|16-Sep-12|           0|       44|      2|              17|       20|            15|
|17-Sep-12|           0|       11|      5|               0|        2|             2|
|18-Sep-12|           0|       28|     18|               3|      

In [0]:
from pyspark.sql.functions import col, when

# Replace invalid or null values
uber = uber.withColumn("Completed Trips", when(col("Completed Trips").isNull(), 0).otherwise(col("Completed Trips").cast("int")))

# Verify column names and clean data
uber.show(5)

# Group the data by date and sum completed trips
completed_trips_by_date = uber.groupBy("Date").sum("Completed Trips")
completed_trips_by_date.show()

# Find the date with most completed trips
if completed_trips_by_date.count() > 0:
    date_with_most_completed_trips = completed_trips_by_date.orderBy(col("sum(Completed Trips)").desc()).first()
    if date_with_most_completed_trips:
        print(f"Date with most completed trips: {date_with_most_completed_trips['Date']} with {date_with_most_completed_trips['sum(Completed Trips)']} trips")
    else:
        print("No data available for completed trips.")
else:
    print("No data available after grouping by date.")

+---------+------------+--------+------+---------------+--------+--------------+
|     Date|Time (Local)|Eyeballs|Zeroes|Completed Trips|Requests|Unique Drivers|
+---------+------------+--------+------+---------------+--------+--------------+
|10-Sep-12|           7|       5|     0|              2|       2|             9|
|     null|           8|       6|     0|              2|       2|            14|
|     null|           9|       8|     3|              0|       0|            14|
|     null|          10|       9|     2|              0|       1|            14|
|     null|          11|      11|     1|              4|       4|            11|
+---------+------------+--------+------+---------------+--------+--------------+
only showing top 5 rows

+---------+--------------------+
|     Date|sum(Completed Trips)|
+---------+--------------------+
|11-Sep-12|                   1|
|13-Sep-12|                   0|
|17-Sep-12|                   0|
|18-Sep-12|                   3|
|21-Sep-12|    